# 04. Recommend Courses to Employees

This exercise builds a model that predicts the rating, a given employee will provide for a given course. We then use this model to identify courses that the employee would prefer most



In [ ]:
#Install all related packages. If you find additional packages missing, please follow the same technique.
#If you are not using anaconda, then use pip to install the same packages

import sys
!conda install --yes --prefix {sys.prefix} pandas tensorflow scikit-learn

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3/envs/hr

  added / updated specs:
    - pandas
    - scikit-learn
    - tensorflow


The following NEW packages will be INSTALLED:

  joblib             pkgs/main/noarch::joblib-0.15.1-py_0
  llvm-openmp        pkgs/main/osx-64::llvm-openmp-10.0.0-h28b9765_0
  scikit-learn       pkgs/main/osx-64::scikit-learn-0.22.1-py37h27c97d8_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 04.04. Prepare for Embedding

In [3]:
#Loading data

from csv import reader
import pandas as pd
import os
import numpy as np
import math

ratings_data = pd.read_csv("/content/drive/MyDrive/Ai in HR/employee_course_ratings.csv")

ratings_data.head()

,EmployeeID,EmpName,CourseID,CourseName,Rating
0,1408,Ignace Ormonde,14,Video Production,3
1,1249,Gabriela Balcon,17,Translation,2
2,1158,Enrique Lewer,8,IT Architecture,3
3,1564,Wallie Byrd,18,Natural Language Processing,3
4,1334,Hannah Ganter,6,Java Programming,4


In [4]:
#Build list of unique Employees
emp_list=ratings_data.groupby(
    ['EmployeeID','EmpName']).size().reset_index()
emp_list.head()
print("Total Employees: ",len(emp_list))

#Build list of unique Courses
course_list=ratings_data.groupby(
    ['CourseID','CourseName']).size().reset_index()
course_list.head()
print("Total Courses: ", len(course_list))

Total Employees:  638
Total Courses:  25


In [5]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

#build employee embedding vector
#we are using IDs a the direct index to embedding.
#Since IDs are continuous, we dont need ID-name mapping.
#We can also build a vocabulary alternatively.

emp_input = Input(shape=[1], name="Emp-Input")
emp_embed = Embedding(2001,  #max value of employee ID
                      5,
                      name="Emp-Embedding")(emp_input)
emp_vec = Flatten(name="Emp-Flatten")(emp_embed)

#build course embedding vector
course_input = Input(shape=[1],name="Course-Input")
course_embed = Embedding(len(course_list) + 1,
                         5,
                         name="Course-Embedding")(course_input)
course_vec = Flatten(name="Course-Flatten")(course_embed)

#merge the vectors
merged_vec = Concatenate()([emp_vec,course_vec])

## 04.05. Building the Keras Rating Model

The recommendation works as follows

1. Build a model that can predict the rating, a given employee may give to a course he/she has not taken so far
2. Use the model to predict possible ratings for all courses, for this employee.
3. Recommend the courses that have the top predicted ratings

In [6]:
#Given an Employee and a Course, this model predicts the 
#rating the employee will give this couse

from sklearn.model_selection import train_test_split

ratings_train, ratings_test = train_test_split(
                                ratings_data, test_size=0.1)

#add fully connected layers
fc_layer1 = Dense(128,activation="relu")(merged_vec)
fc_layer2 = Dense(32, activation="relu")(fc_layer1)
model_output = Dense(1)(fc_layer2)

rating_model= Model([emp_input,course_input],model_output)

rating_model.compile(optimizer="adam",
                     loss="mean_squared_error")

rating_model.summary()

print("Fitting the model:")
#Fit the model
model_fit = rating_model.fit(
    x=[ratings_train.EmployeeID, ratings_train.CourseID],
    y=ratings_train.Rating,
    epochs=25,
    verbose=1,
    validation_split=0.1
    )

print("Evaluating the model:")
rating_model.evaluate(
    x=[ratings_test.EmployeeID, ratings_test.CourseID],
    y=ratings_test.Rating)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Emp-Input (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
Course-Input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
Emp-Embedding (Embedding)       (None, 1, 5)         10005       Emp-Input[0][0]                  
__________________________________________________________________________________________________
Course-Embedding (Embedding)    (None, 1, 5)         130         Course-Input[0][0]               
______________________________________________________________________________________________

2.3234028816223145

## 04.06. Recommending Courses with Keras

In [7]:
#Predicting the Rating for a given employee and a course
#for employee 1029 and course 8

rating_model.predict(
    [pd.Series([1029]),
     pd.Series([8])])

array([[2.6903768]], dtype=float32)

In [8]:
emp_to_predict="Harriot Laflin"

#Get employee ID for the employee name
pred_emp_id=emp_list[emp_list['EmpName'] == emp_to_predict]["EmployeeID"].iloc[0]

#find Courses already taken by employee. We dont want to predict those.
completed_courses=ratings_data[
                    ratings_data["EmployeeID"] == pred_emp_id]["CourseID"].unique()

#Courses not taken by employee
new_courses = course_list.query("CourseID not in @completed_courses")["CourseID"]

#Create a list with the same employee ID repeated for the same number of times as the
#number of new courses. This provides the employee and course Series with same size
emp_dummy_list=pd.Series(np.array([pred_emp_id for i in range(len(new_courses))]))

#Predict ratings for the new courses for this employee
projected_ratings = rating_model.predict([emp_dummy_list,new_courses])
flat_ratings = np.array([x[0] for x in projected_ratings])

print("Course Ratings: ", flat_ratings)

#Recommend top 5 courses
print("\nRating  CourseID CourseName\n-----------------------------------")
for idx in (-flat_ratings).argsort()[:5]:
    course_id=new_courses.iloc[idx]
    course_name=course_list.query("CourseID == @course_id")["CourseName"].iloc[0]
    print(" ", round(flat_ratings[idx],1),"    ", course_id, "   ", course_name)



Course Ratings:  [3.1062288 2.900057  3.8283718 4.5638146 3.9206812 4.509118  3.930762
 3.9526196 4.3587475 4.5933003 3.9396985 3.5021565 3.7930236 2.8583014
 4.5471163 4.354575  3.3472378 3.3131287 3.5518162 4.018592  3.6649632
 3.5773735 3.5881054]

Rating  CourseID CourseName
-----------------------------------
  4.6      11     Analytical Reasoning
  4.6      5     HelpDesk
  4.5      16     Audio Production
  4.5      7     Project Management
  4.4      10     Mobile Development
